In [ ]:
pip install mysql-connector-python

In [44]:
import numpy as np
import pandas as pd
import mysql.connector

In [ ]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='1408',
    database='cts'
)

cursor = conn.cursor()

# Insert training data (65 columns) into train_data table
for i, row in df_train.iterrows():
    values = tuple(row.values)
    placeholders = ",".join(['%s'] * len(values))
    sql = f"INSERT INTO train_data VALUES ({placeholders})"
    cursor.execute(sql, values)
conn.commit()

# Insert test data (56 columns) into test_data table
for i, row in df_test.iterrows():
    values = tuple(row.values)
    placeholders = ",".join(['%s'] * len(values))
    sql = f"INSERT INTO test_data VALUES ({placeholders})"  # Different table!
    cursor.execute(sql, values)
conn.commit()

cursor.close()
conn.close()


In [45]:
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='1408',
    database='cts'
)
cursor = conn.cursor()

cursor.execute("SELECT * FROM train_data")
train_results = cursor.fetchall()


cursor.execute("DESCRIBE train_data")
train_columns = [column[0] for column in cursor.fetchall()]

df_train = pd.DataFrame(train_results, columns=train_columns)

# Load test data
cursor.execute("SELECT * FROM test_data")
test_results = cursor.fetchall()

# Get column names for test data
cursor.execute("DESCRIBE test_data")
test_columns = [column[0] for column in cursor.fetchall()]

# Create DataFrame for test data
df_test = pd.DataFrame(test_results, columns=test_columns)

cursor.close()
conn.close()

In [46]:
print(f"Training data shape: {df_train.shape}")
print(f"Test data shape: {df_test.shape}")

Training data shape: (8000, 65)
Test data shape: (500, 56)


In [47]:
print("Missing values in training data:", df_train.isnull().sum().sum())
print("Missing values in test data:", df_test.isnull().sum().sum())

Missing values in training data: 0
Missing values in test data: 0


In [74]:
colu = ['Component1_fraction', 'Component2_fraction', 'Component3_fraction',
       'Component4_fraction', 'Component5_fraction', 'Component1_Property1',
       'Component2_Property1', 'Component3_Property1', 'Component4_Property1',
       'Component5_Property1', 'Component1_Property2', 'Component2_Property2',
       'Component3_Property2', 'Component4_Property2', 'Component5_Property2',
       'Component1_Property3', 'Component2_Property3', 'Component3_Property3',
       'Component4_Property3', 'Component5_Property3', 'Component1_Property4',
       'Component2_Property4', 'Component3_Property4', 'Component4_Property4',
       'Component5_Property4', 'Component1_Property5', 'Component2_Property5',
       'Component3_Property5', 'Component4_Property5', 'Component5_Property5',
       'Component1_Property6', 'Component2_Property6', 'Component3_Property6',
       'Component4_Property6', 'Component5_Property6', 'Component1_Property7',
       'Component2_Property7', 'Component3_Property7', 'Component4_Property7',
       'Component5_Property7', 'Component1_Property8', 'Component2_Property8',
       'Component3_Property8', 'Component4_Property8', 'Component5_Property8',
       'Component1_Property9', 'Component2_Property9', 'Component3_Property9',
       'Component4_Property9', 'Component5_Property9', 'Component1_Property10',
       'Component2_Property10', 'Component3_Property10',
       'Component4_Property10', 'Component5_Property10', 'BlendProperty1',
       'BlendProperty2', 'BlendProperty3', 'BlendProperty4', 'BlendProperty5',
       'BlendProperty6', 'BlendProperty7', 'BlendProperty8', 'BlendProperty9',
       'BlendProperty10']

In [97]:
outlier_counts = {}

for col in colu:
    Q1 = df_train[col].quantile(0.25)
    Q3 = df_train[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Count outliers
    outliers = df_train[(df_train[col] < lower_bound) | (df_train[col] > upper_bound)]
    outlier_counts[col] = outliers.shape[0]

print(outlier_counts)


{'Component1_fraction': 0, 'Component2_fraction': 0, 'Component3_fraction': 0, 'Component4_fraction': 0, 'Component5_fraction': 0, 'Component1_Property1': 0, 'Component2_Property1': 0, 'Component3_Property1': 0, 'Component4_Property1': 0, 'Component5_Property1': 0, 'Component1_Property2': 0, 'Component2_Property2': 0, 'Component3_Property2': 0, 'Component4_Property2': 0, 'Component5_Property2': 0, 'Component1_Property3': 0, 'Component2_Property3': 0, 'Component3_Property3': 0, 'Component4_Property3': 0, 'Component5_Property3': 0, 'Component1_Property4': 0, 'Component2_Property4': 0, 'Component3_Property4': 0, 'Component4_Property4': 0, 'Component5_Property4': 0, 'Component1_Property5': 0, 'Component2_Property5': 0, 'Component3_Property5': 0, 'Component4_Property5': 0, 'Component5_Property5': 0, 'Component1_Property6': 0, 'Component2_Property6': 0, 'Component3_Property6': 0, 'Component4_Property6': 0, 'Component5_Property6': 0, 'Component1_Property7': 0, 'Component2_Property7': 0, 'Comp

In [98]:
for col in colu:
    Q1 = df_train[col].quantile(0.25)
    Q3 = df_train[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Keep only rows within bounds
    df_train = df_train[(df_train[col] >= lower_bound) & (df_train[col] <= upper_bound)]

In [ ]:
X_train = df_train.iloc[:, :55]
y_train = df_train.iloc[:, 55:] 

In [100]:
def simple_outlier_check(df):
    print("=== SIMPLE OUTLIER CHECK - ALL COLUMNS ===")
    
    outlier_count = 0
    columns_with_outliers = 0
    total_features = len(df.columns)
    
    print(f"Checking all {total_features} columns for outliers...\n")
    
    for column in df.columns:  
        Q1 = df[column].quantile(0.25)
        Q3 = df[column].quantile(0.75)
        IQR = Q3 - Q1
        
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        outliers = len(df[(df[column] < lower_bound) | (df[column] > upper_bound)])
        
        if outliers > 0:
            print(f"{column}: {outliers} outliers")
            outlier_count += outliers
            columns_with_outliers += 1
    
    print(f"\n=== SUMMARY ===")
    print(f"Total columns checked: {total_features}")
    print(f"Columns with outliers: {columns_with_outliers}")
    print(f"Total outlier data points: {outlier_count}")
    print(f"Percentage of outlier data points: {(outlier_count/(len(df) * total_features))*100:.2f}%")
    
    if outlier_count > 0:
        print("✓ Outliers detected - consider handling them")
    else:
        print("✓ No outliers detected in any column")

# Check for outliers in ALL columns
simple_outlier_check(X_train)


=== SIMPLE OUTLIER CHECK - ALL COLUMNS ===
Checking all 55 columns for outliers...


=== SUMMARY ===
Total columns checked: 55
Columns with outliers: 0
Total outlier data points: 0
Percentage of outlier data points: 0.00%
✓ No outliers detected in any column


In [101]:
from sklearn.model_selection import train_test_split

# Assuming X_train_capped is your preprocessed features and y_train your targets
X_train, X_val, y_train_split, y_val = train_test_split(
    X_train, y_train,
    test_size=0.3,
    random_state=42
)


In [141]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor


model = MultiOutputRegressor(RandomForestRegressor(random_state=42, n_estimators=100))
model.fit(X_train, y_train_split)  # Your preprocessed training data




MultiOutputRegressor(estimator=RandomForestRegressor(random_state=42))

In [142]:

for i, target in enumerate(y_train.columns):
    print(f"\nFeature importance for {target}:")
    rf = model.estimators_[i]   # Access underlying Random Forest for this target
    importances = pd.Series(rf.feature_importances_, index=X_train.columns)
    importances = importances.sort_values(ascending=False)
    print(importances.head(10))  


Feature importance for BlendProperty1:
Component5_fraction     0.433833
Component2_fraction     0.199907
Component4_fraction     0.053774
Component3_Property1    0.046271
Component4_Property1    0.043918
Component1_Property1    0.024141
Component5_Property1    0.023912
Component2_Property1    0.022695
Component1_fraction     0.018072
Component3_fraction     0.013532
dtype: float64

Feature importance for BlendProperty2:
Component5_fraction     0.425988
Component3_fraction     0.103337
Component2_fraction     0.101448
Component4_Property2    0.082340
Component5_Property2    0.048174
Component3_Property2    0.045793
Component4_fraction     0.036613
Component1_fraction     0.023067
Component1_Property2    0.014656
Component2_Property2    0.007091
dtype: float64

Feature importance for BlendProperty3:
Component2_fraction     0.711480
Component2_Property7    0.052498
Component1_fraction     0.051422
Component1_Property7    0.039188
Component3_Property7    0.034957
Component2_Property8    0

In [147]:
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# Predict on validation set
y_val_pred = model.predict(X_val)

# Calculate metrics
mse = mean_squared_error(y_val, y_val_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_val, y_val_pred)

print(f"Validation MSE: {mse:.4f}")
print(f"Validation RMSE: {rmse:.4f}")
print(f"Validation R²: {r2:.4f}")


Validation MSE: 0.0104
Validation RMSE: 0.1018
Validation R²: 0.9891


In [148]:
import numpy as np

def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    # Avoid division by zero
    non_zero_idx = y_true != 0
    return np.mean(np.abs((y_true[non_zero_idx] - y_pred[non_zero_idx]) / y_true[non_zero_idx])) * 100

# Predictions on validation set
y_val_pred = model.predict(X_val)

# Calculate MAPE
mape = mean_absolute_percentage_error(y_val, y_val_pred)

print("Validation MAPE: {:.2f}%".format(mape))


Validation MAPE: 36.71%


Actual: [[-0.457835  -0.421137  -0.0129449 -0.557746  -0.963275  -0.863224
  -0.0225749  0.118969  -0.703928  -0.092791 ]]
Predicted: [[-0.33539096 -0.29769796 -1.38298225 -0.35418145 -0.7304854  -0.2243001
  -1.364158   -1.29754434 -0.15506874  0.02494669]]


TEST DATA

In [154]:
df_test.columns

Index(['ID', 'Component1_fraction', 'Component2_fraction',
       'Component3_fraction', 'Component4_fraction', 'Component5_fraction',
       'Component1_Property1', 'Component2_Property1', 'Component3_Property1',
       'Component4_Property1', 'Component5_Property1', 'Component1_Property2',
       'Component2_Property2', 'Component3_Property2', 'Component4_Property2',
       'Component5_Property2', 'Component1_Property3', 'Component2_Property3',
       'Component3_Property3', 'Component4_Property3', 'Component5_Property3',
       'Component1_Property4', 'Component2_Property4', 'Component3_Property4',
       'Component4_Property4', 'Component5_Property4', 'Component1_Property5',
       'Component2_Property5', 'Component3_Property5', 'Component4_Property5',
       'Component5_Property5', 'Component1_Property6', 'Component2_Property6',
       'Component3_Property6', 'Component4_Property6', 'Component5_Property6',
       'Component1_Property7', 'Component2_Property7', 'Component3_Property

In [155]:
cols = ['ID', 'Component1_fraction', 'Component2_fraction',
       'Component3_fraction', 'Component4_fraction', 'Component5_fraction',
       'Component1_Property1', 'Component2_Property1', 'Component3_Property1',
       'Component4_Property1', 'Component5_Property1', 'Component1_Property2',
       'Component2_Property2', 'Component3_Property2', 'Component4_Property2',
       'Component5_Property2', 'Component1_Property3', 'Component2_Property3',
       'Component3_Property3', 'Component4_Property3', 'Component5_Property3',
       'Component1_Property4', 'Component2_Property4', 'Component3_Property4',
       'Component4_Property4', 'Component5_Property4', 'Component1_Property5',
       'Component2_Property5', 'Component3_Property5', 'Component4_Property5',
       'Component5_Property5', 'Component1_Property6', 'Component2_Property6',
       'Component3_Property6', 'Component4_Property6', 'Component5_Property6',
       'Component1_Property7', 'Component2_Property7', 'Component3_Property7',
       'Component4_Property7', 'Component5_Property7', 'Component1_Property8',
       'Component2_Property8', 'Component3_Property8', 'Component4_Property8',
       'Component5_Property8', 'Component1_Property9', 'Component2_Property9',
       'Component3_Property9', 'Component4_Property9', 'Component5_Property9',
       'Component1_Property10', 'Component2_Property10',
       'Component3_Property10', 'Component4_Property10',
       'Component5_Property10']

In [156]:
for col in cols:
    Q1 = df_test[col].quantile(0.25)
    Q3 = df_test[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Keep only rows within bounds
    df_test = df_test[(df_test[col] >= lower_bound) & (df_test[col] <= upper_bound)]

In [157]:
#id test data
test_ids = df_test.iloc[:, 0]        
X_test = df_test.iloc[:, 1:56]

In [158]:
def simple_outlier_check(df):
    print("=== SIMPLE OUTLIER CHECK - ALL COLUMNS ===")
    
    outlier_count = 0
    columns_with_outliers = 0
    total_features = len(df.columns)
    
    print(f"Checking all {total_features} columns for outliers...\n")
    
    for column in df.columns:  
        Q1 = df[column].quantile(0.25)
        Q3 = df[column].quantile(0.75)
        IQR = Q3 - Q1
        
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        outliers = len(df[(df[column] < lower_bound) | (df[column] > upper_bound)])
        
        if outliers > 0:
            print(f"{column}: {outliers} outliers")
            outlier_count += outliers
            columns_with_outliers += 1
    
    print(f"\n=== SUMMARY ===")
    print(f"Total columns checked: {total_features}")
    print(f"Columns with outliers: {columns_with_outliers}")
    print(f"Total outlier data points: {outlier_count}")
    print(f"Percentage of outlier data points: {(outlier_count/(len(df) * total_features))*100:.2f}%")
    
    if outlier_count > 0:
        print("✓ Outliers detected - consider handling them")
    else:
        print("✓ No outliers detected in any column")

# Check for outliers in ALL columns
simple_outlier_check(X_test)


=== SIMPLE OUTLIER CHECK - ALL COLUMNS ===
Checking all 55 columns for outliers...


=== SUMMARY ===
Total columns checked: 55
Columns with outliers: 0
Total outlier data points: 0
Percentage of outlier data points: 0.00%
✓ No outliers detected in any column


In [159]:
y_test_pred = model.predict(X_test)
y_test_pred

array([[-0.33539096, -0.29769796, -1.38298225, ..., -1.29754434,
        -0.15506874,  0.02494669],
       [ 1.13325051,  0.44971822,  0.43135744, ...,  1.5127998 ,
        -0.44302033,  1.77983608],
       [-0.52821147, -0.82557997,  0.80018589, ..., -0.35625047,
        -0.80918889,  0.70699536],
       ...,
       [-0.19610957, -0.52177026,  0.45689924, ..., -0.15064181,
        -0.60748019,  0.69876071],
       [ 1.55593071,  0.97481212,  0.01017432, ...,  0.66380964,
         0.11527924, -0.33345959],
       [-0.9878801 , -1.44202996, -2.0471786 , ..., -1.69212483,
        -2.0231749 , -0.21097755]])

In [160]:
pred_df = pd.DataFrame(y_test_pred, columns=y_train.columns)
pred_df.insert(0, 'ID', df_test['ID'])  # Assuming test DataFrame has ID column

In [161]:
pred_df

,ID,BlendProperty1,BlendProperty2,BlendProperty3,BlendProperty4,BlendProperty5,BlendProperty6,BlendProperty7,BlendProperty8,BlendProperty9,BlendProperty10
0,NaN,-0.335391,-0.297698,-1.382982,-0.354181,-0.730485,-0.224300,-1.364158,-1.297544,-0.155069,0.024947
1,2.0,1.133251,0.449718,0.431357,1.037367,1.143091,1.311886,0.416391,1.512800,-0.443020,1.779836
2,3.0,-0.528211,-0.825580,0.800186,-0.225303,1.122039,-0.713156,0.745778,-0.356250,-0.809189,0.706995
3,NaN,-1.324685,-1.483058,0.133084,-1.286258,0.006344,-0.820595,0.125785,-0.215604,-0.831574,0.133072
4,5.0,0.636933,0.499727,-0.448596,0.216395,-0.112382,0.643518,-0.444557,0.208265,0.810735,0.301267
...,...,...,...,...,...,...,...,...,...,...,...
325,326.0,1.041355,0.569298,-0.045851,1.053508,-1.718586,0.749969,-0.052821,0.757304,0.297267,1.043995
326,NaN,-0.755281,-0.539684,0.298348,-1.041196,1.148809,0.183444,0.375951,-0.682531,-0.412114,0.491023
327,NaN,-0.196110,-0.521770,0.456899,-0.629567,1.111113,-0.157535,0.466930,-0.150642,-0.607480,0.698761
328,NaN,1.555931,0.974812,0.010174,0.245813,0.009438,0.482312,-0.026958,0.663810,0.115279,-0.333460


In [162]:
pred_df.to_csv('fuel_blend_predictions.csv', index=False)
print("Test predictions saved to 'fuel_blend_predictions.csv'.")

Test predictions saved to 'fuel_blend_predictions.csv'.


In [ ]:
import joblib

In [ ]:
import joblib
joblib.dump(model, "random_forest_multioutput.pkl")